In [1]:
import numpy as np 
import pandas as pd 
from umap import UMAP 
import leidenalg
import igraph as ig
import math
from sklearn.base import BaseEstimator
from sklearn.metrics import silhouette_score
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN, OPTICS
from sklearn.mixture import GaussianMixture
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import TSNE
import itertools

from lifelines import KaplanMeierFitter, CoxPHFitter
from lifelines.statistics import multivariate_logrank_test
import scipy.stats as stats
import warnings
warnings.filterwarnings('ignore')

/cellar/users/dhalmos/miniconda3/envs/nest/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def leiden_cluster(X, k=25):
    """
    Create a graph from nearest neighbors and find clusters using Leiden algorithm
    
    Parameters:
    -----------
    X : array-like
        The dimensionality reduced data
    k : int, default=25
        Number of nearest neighbors
        
    Returns:
    --------
    g : igraph.Graph
        The created graph
    partition : leidenalg.VertexPartition
        The partition result from Leiden algorithm
    """
    # Find nearest neighbors
    neighbors = NearestNeighbors(n_neighbors=k).fit(X)
    distances, indices = neighbors.kneighbors(X)

    # Build edge list with weighted edges
    edges = []
    weights = []
    num_points = X.shape[0]

    for i in range(num_points):
        for idx, j in enumerate(indices[i]):
            if i == j: 
                continue
            if (j, i) in edges:
                continue
            d = distances[i, idx]
            weight = math.exp(-d)
            edges.append((i, j))
            weights.append(weight)

    # Create an igraph Graph, add vertices and edges
    g = ig.Graph()
    g.add_vertices(num_points)
    g.add_edges(edges)

    # Set the edge attribute 'weight' for our weighted graph
    g.es['weight'] = weights

    # Find partition using Leiden algorithm
    partition = leidenalg.find_partition(g, leidenalg.ModularityVertexPartition)
    #print("Clusters:", partition)
    
    return g, partition


In [3]:
class TSNEClusteringEvaluator(BaseEstimator):
    def __init__(self,  n_components = 2, perplexity = 30.0, early_exaggeration = 12.0, metric = 'euclidean', method = 'barnes_hut'):
        """
        Initialize UMAP and store hyperparameters for later grid search.
        """
        self.n_components = n_components
        self.perplexity=perplexity
        self.early_exaggeration = early_exaggeration
        self.metric = metric
        self.method = method

    def fit_transform(self, X, y=None):
        """
        Fit the UMAP model on the data.
        """
        self.tsne_model = TSNE(n_components =self.n_components, perplexity = self.perplexity, early_exaggeration = self.early_exaggeration, metric = self.metric, method = self.method, random_state = 42, n_jobs = 8)
        y = self.tsne_model.fit_transform(X)
        return y

    def score(self, X, y=None):
        """
        Transform the data into the reduced space and then run a set of clustering algorithms.
        Compute the silhouette score for each (if valid) and return the average silhouette score.
        """
        X_reduced = self.fit_transform(X)
        scores = []  # To collect silhouette scores
        
        # List of clustering algorithms to evaluate.
        # You can adjust these or add new ones as needed.
        clustering_methods = [
            ('KMeans', KMeans()), # 6 k means works the best 
            ('Agglomerative', AgglomerativeClustering()),
            ('Spectral', SpectralClustering()),
            ('DBSCAN', DBSCAN()),
            ('GaussianMixture', GaussianMixture()),
            ('Leiden', leiden_cluster(X_reduced)) # Custom Leiden clustering,
        ]
        
        for name, algorithm in clustering_methods:
            try:
                # Obtain cluster labels.
                # Some algorithms have fit_predict, others require separate fitting and predicting.
                if name in ['KMeans', 'Agglomerative', 'Spectral', 'DBSCAN', 'OPTICS']:
                    labels = algorithm.fit_predict(X_reduced)
                elif name == 'GaussianMixture':
                    algorithm.fit(X_reduced)
                    labels = algorithm.predict(X_reduced)
                elif name == 'Leiden':
                    g, partition = leiden_cluster(X_reduced)
                    labels = np.array(partition.membership)
                
                # Check if we have at least two clusters.
                # For DBSCAN, we exclude noise labeled as -1.
                if name == 'DBSCAN':
                    valid_idx = labels != -1
                    if len(np.unique(labels[valid_idx])) < 2:
                        continue
                    score = silhouette_score(X_reduced[valid_idx], labels[valid_idx])
                else:
                    if len(np.unique(labels)) < 2:
                        continue  # Skip if only one cluster is produced.
                    score = silhouette_score(X_reduced, labels)
                
                scores.append(score)
            except Exception as e:
                # In a production system you might log errors; here we just print them.
                print(f"Error with {name}: {e}")
                continue

        # If none of the clustering methods produced a valid silhouette score,
        # return a default low score. Otherwise, return the average.
        if scores:
            return np.mean(scores)
        return -1.0  # or another value indicating failure


In [4]:
gene_var_list = ['1000', '2000', '5000', '10000']
top_variance_dict = {gene : pd.read_csv(f'../../Data/processed/top{gene}genesByVariance.txt', sep = '\t', header = None) for gene in gene_var_list}

In [5]:
top_variance_dict

{'1000':                 0             1
 0      KRT14|3861  1.997217e+11
 1      KRT13|3860  1.353050e+11
 2      KRT6A|3853  9.710513e+10
 3      KRT16|3868  9.563024e+10
 4     SMR3B|10879  8.893915e+10
 ..            ...           ...
 995   MAGEA4|4103  6.462169e+06
 996  PLA2G2A|5320  6.451723e+06
 997     CES2|8824  6.450381e+06
 998     CCL5|6352  6.444331e+06
 999   SNRPD2|6633  6.427743e+06
 
 [1000 rows x 2 columns],
 '2000':                    0             1
 0         KRT14|3861  1.997217e+11
 1         KRT13|3860  1.353050e+11
 2         KRT6A|3853  9.710513e+10
 3         KRT16|3868  9.563024e+10
 4        SMR3B|10879  8.893915e+10
 ...              ...           ...
 1995      LMAN1|3998  1.774504e+06
 1996   PPP1R12B|4660  1.774193e+06
 1997  PPP1R14B|26472  1.771483e+06
 1998     ARPC5|10092  1.769507e+06
 1999     COX6A2|1339  1.768273e+06
 
 [2000 rows x 2 columns],
 '5000':                    0             1
 0         KRT14|3861  1.997217e+11
 1         KRT13|386

In [6]:
df_log = pd.read_csv('../../Data/processed/TCGA.HNSC.expression_log_zscore_all.txt', sep = '\t')

In [15]:
df_log

,patient_id,sample_id,?|100133144,?|100134869,?|10357,?|10431,?|155060,?|340602,?|388795,?|391343,...,ZWILCH|55055,ZWINT|11130,ZXDA|7789,ZXDB|158586,ZXDC|79364,ZYG11A|440590,ZYG11B|79699,ZYX|7791,ZZEF1|23140,ZZZ3|26009
0,TCGA-4P-AA8J,TCGA-4P-AA8J-01A-11R-A39I-07,-1.106245,0.361294,-0.789756,0.976317,1.362804,0.953466,1.297058,-0.347199,...,-1.525366,-0.274022,-0.526428,-1.169339,-1.119308,-0.231705,-1.126895,2.000291,-0.792082,-1.872335
1,TCGA-BA-4074,TCGA-BA-4074-01A-01R-1436-07,0.378257,0.251161,1.677288,1.885588,-1.400457,-0.397433,-0.600539,-0.347199,...,3.022373,0.652772,-0.578960,-0.867707,-3.033171,-0.472145,0.477495,-1.501729,-3.894482,1.496986
2,TCGA-BA-4075,TCGA-BA-4075-01A-01R-1436-07,0.329367,-1.112774,1.761005,1.108671,-0.422703,-0.397433,-0.032339,-0.347199,...,0.705112,-0.328989,-0.803753,-0.870504,-1.891197,-0.933606,-0.754044,0.461025,-3.581890,1.179483
3,TCGA-BA-4076,TCGA-BA-4076-01A-01R-1436-07,0.344664,-0.963293,1.210642,0.460375,0.109229,-0.397433,0.947597,-0.347199,...,-0.330099,0.404923,0.582490,0.538005,0.861776,0.645006,0.211680,-1.362640,-1.477910,1.235520
4,TCGA-BA-4077,TCGA-BA-4077-01B-01R-1436-07,0.342013,0.229375,1.568010,-0.588324,-1.111551,-0.397433,1.168137,-0.347199,...,1.952492,1.177951,0.490112,-0.419747,0.825191,0.885116,0.371009,1.157012,-1.156172,0.126140
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,TCGA-UF-A7JV,TCGA-UF-A7JV-01A-11R-A34R-07,-0.996082,0.482659,-0.480007,0.179477,2.189234,-0.397433,1.716465,-0.347199,...,0.053321,0.336707,-0.638038,-0.186011,-0.742190,-1.166635,-0.288716,1.536669,-1.285720,-0.279166
541,TCGA-UP-A6WW,TCGA-UP-A6WW-01A-12R-A34R-07,0.166049,0.737227,-1.125382,0.851566,0.735091,-0.397433,-0.638541,0.839493,...,0.606629,1.757558,-0.472822,-1.075684,-1.059174,1.978971,-1.137601,0.213409,-0.205424,-0.968876
542,TCGA-WA-A7GZ,TCGA-WA-A7GZ-01A-11R-A34R-07,1.861002,1.713751,0.062277,-1.528399,0.977881,0.075130,1.298651,-0.347199,...,-0.030937,0.872259,0.248617,-0.070527,0.736580,-0.612015,-0.461319,-0.732398,1.063576,0.206015
543,TCGA-WA-A7GZ,TCGA-WA-A7GZ-11A-11R-A34R-07,2.109506,2.312825,-1.598348,-0.443611,1.529241,-0.397433,-1.060700,0.765167,...,-1.648004,-1.528002,1.128967,0.505933,0.532465,0.821300,3.471063,-2.633568,1.189505,-0.284683


In [7]:
gene_count_input = {gene : df_log[top_variance_dict[gene][0].values].values for gene in gene_var_list}

In [36]:
# Define parameter grid for UMAP hyperparameters.
param_grid = {
    'early_exaggeration': [5, 10, 50],
    'perplexity': [5, 10, 50],
    'n_components': [2],
    'metric' : ['cosine'],
    'method' : ['barnes_hut'],
    'dataset' : gene_var_list
}

grid_search = list(itertools.product(*[param_grid['n_components'], param_grid['perplexity'], param_grid['early_exaggeration'], param_grid['metric'], param_grid['method'], param_grid['dataset']]))

In [37]:
best_score = float('-inf')
best_params = None
for param_set in grid_search:
    score = TSNEClusteringEvaluator(*[param for param in param_set[:-1]]).score(gene_count_input[param_set[-1]])
    print(param_set, score)
    if score > best_score:
        best_score = score
        best_params = param_set

(2, 5, 5, 'cosine', 'barnes_hut', '1000') 0.24275021
(2, 5, 5, 'cosine', 'barnes_hut', '2000') 0.29365075
(2, 5, 5, 'cosine', 'barnes_hut', '5000') 0.2524561
(2, 5, 5, 'cosine', 'barnes_hut', '10000') 0.2156702
(2, 5, 10, 'cosine', 'barnes_hut', '1000') 0.22611716
(2, 5, 10, 'cosine', 'barnes_hut', '2000') 0.26269957
(2, 5, 10, 'cosine', 'barnes_hut', '5000') 0.2739103
(2, 5, 10, 'cosine', 'barnes_hut', '10000') 0.27849233
(2, 5, 50, 'cosine', 'barnes_hut', '1000') 0.23627827
(2, 5, 50, 'cosine', 'barnes_hut', '2000') 0.18140602
(2, 5, 50, 'cosine', 'barnes_hut', '5000') 0.18090612
(2, 5, 50, 'cosine', 'barnes_hut', '10000') 0.22313583
(2, 10, 5, 'cosine', 'barnes_hut', '1000') 0.21305177
(2, 10, 5, 'cosine', 'barnes_hut', '2000') 0.22289804
(2, 10, 5, 'cosine', 'barnes_hut', '5000') 0.32359427
(2, 10, 5, 'cosine', 'barnes_hut', '10000') 0.2652227
(2, 10, 10, 'cosine', 'barnes_hut', '1000') 0.23587564
(2, 10, 10, 'cosine', 'barnes_hut', '2000') 0.1997295
(2, 10, 10, 'cosine', 'barnes_h

Ones to eye
best_params = (2, 10, 12, 'cosine', 'barnes_hut', '1000')

In [38]:
best_params = (2, 10, 12, 'cosine', 'barnes_hut', '1000')
output = TSNEClusteringEvaluator(*[param for param in best_params[:-1]]).fit_transform(gene_count_input[best_params[-1]])

In [39]:
output[:, 0].shape

(545,)

In [40]:
df = pd.DataFrame({'Sample' : df_log['sample_id'], 0 : output[:, 0],1 : output[:, 1]})

In [41]:
df.to_csv('best_tsne_params.csv', sep = '\t', index = False)